In [1]:
import pandas as pd
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil
from function import *

In [11]:
#全体の流れは、function.pyのmake_objに記載の通り
#ここでは、処理を途中でやめること、darkがない場合も考慮した方法で書いていく。

In [6]:
filelist = glob.glob(r'/mnt/e/2003/COMA*.fits')
for obsfile in filelist[2700:2702]:

        dark_1 = make1Expdark(obsfile)
        obsheader = readheader(obsfile)
        Q_CHEB = obsheader["Q_CHEB"]
        dark_CHEB = dark_1 * int(Q_CHEB)
        # obsdata
        # この時点で、RAWをAddに変換する。
        if (obsheader["Q_CHAM"] == 0):
            obsdata_RAW = readdata(obsfile)
            Q_CHCN = obsheader["Q_CHCN"]
            A = np.array(np.split(obsdata_RAW, Q_CHCN, axis=1))
            obsdata = np.sum(A, axis=0)
        elif (obsheader["Q_CHAM"] == 1):
            obsdata = readdata(obsfile)
        elif (obsheader["Q_CHAM"] == 2):
            print("ECOモードによりskip")
            continue

        # skydata作成
        skydata = obsdata - dark_CHEB
        # q_bsepによりposiとnegaにわける
        # Addモードの場合だけでやってみる。
        skydata_p = q_bsep_posi(skydata)
        skydata_n = q_bsep_nega(skydata)

        # z方向平均
        sky_pa = mean_z(skydata_p)
        sky_na = mean_z(skydata_n)

        # gaussian　平均
        sky_paG = gaussfilter(sky_pa)
        sky_naG = gaussfilter(sky_na)

        # Flatの作成
        sky_paF = sky_pa / sky_paG
        sky_naF = sky_na / sky_naG

        comqfile = obsfile.replace("COMA", "COMQ")

        comq_obs = readdata(comqfile)
        obj_obs = q_subch(comq_obs)
        # Flatで割る
        obj_obs_datP0 = obj_obs / sky_naF
        obj_obs_datN0 = obj_obs / sky_paF

        # Exptimeで割る
        obj_obs_datP0_per1sec = obj_obs_datP0 / exptime(comqfile)
        obj_obs_datN0_per1sec = obj_obs_datN0 / exptime(comqfile)

        os.makedirs(r'/mnt/e/'+obsyear(obsfile)+r'/out_obj/', exist_ok=True)
        outputfol_P = r'/mnt/e/' + \
            obsyear(obsfile)+r'/out_obj/obj_' + \
            obsfile[-10:-5]+r'_datP0_per1sec.fits'
        makefits(obj_obs_datP0_per1sec, outputfol_P, comqfile)

        outputfol_N = r'/mnt/e/' + \
            obsyear(obsfile)+r'/out_obj/obj_' + \
            obsfile[-10:-5]+r'_datN0_per1sec.fits'
        makefits(obj_obs_datN0_per1sec, outputfol_N, comqfile)


/mnt/e/2003/COMA00036923.fits


/mnt/e/2003/COMA00036925.fits


In [10]:
def make1Expdark_q(obsfile):
    print(obsfile)
    df = identify(obsfile)
    year = obsyear(obsfile)
    darkbasefolder = r'/mnt/e/' + year + r'/dark/'
    obsdata = readdata(obsfile)
    # obsdataと同じshapeの0のnumpyを作成
    darkdata = np.zeros(list(obsdata.shape))
    if len(df) == 0:
        print("該当のダークが存在しない")
        return None
        # ここにはその場合の処理を記載する。

    # 特定したダークファイル一枚一枚に対して、Q_CHEBが同じものを抽出??

    # df[df["Q_CHEB"].duplicated()]
    for darkfile in df["FRAME_ID"]:
        darkfilepath = darkbasefolder + darkfile + r'.fits'
        darkdata += np.mean(readdata(darkfilepath), axis=1, keepdims=True)
    # meanをとる
    meandark = darkdata / len(df)
    # 1Expあたりに直す
    meandark_1 = meandark / int(df["Q_CHEB"][0])
    return meandark_1

In [14]:
filelist = glob.glob(r'/mnt/e/2003/COMA*.fits')
filelist[0]
A = make1Expdark_q(filelist[0])
if A is None:
    print("Noneです")

/mnt/e/2003/COMA00032397.fits
該当のダークが存在しない
Noneです


In [9]:
filelist = glob.glob(r'/mnt/e/2003/COMA*.fits')
for obsfile in filelist[2630:2670]:
    try:
        make_obj(obsfile)
    
    except Exception as e:
        
        print('=== エラー内容 ===')
        print('type:' + str(type(e)))
        print('args:' + str(e.args))
        print('message:' + e.message)
        print('e自身:' + str(e))

/mnt/e/2003/COMA00036796.fits
該当のダークが存在しない
/mnt/e/2003/COMA00036798.fits
該当のダークが存在しない
/mnt/e/2003/COMA00036800.fits
該当のダークが存在しない
/mnt/e/2003/COMA00036801.fits


ValueError: non-broadcastable output operand with shape (1,2,240,320) doesn't match the broadcast shape (2,2,240,320)

In [4]:
filelist = glob.glob(r'/mnt/e/2003/COMA*.fits')
make1Expdark(filelist[0])
make_obj(filelist[0])

/mnt/e/2003/COMA00032397.fits
該当のダークが存在しない
/mnt/e/2003/COMA00032397.fits
該当のダークが存在しない


In [12]:
#errorlogの取り方
import logging
logging.error()

TypeError: error() missing 1 required positional argument: 'msg'